<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural
## Desafio 1


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".


In [60]:
import numpy as np
import pandas as pd
import random

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [61]:
# --------------------------------------
# Carga del dataset
# --------------------------------------

train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test  = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train = train.data
y_train = train.target
X_test  = test.data
y_test  = test.target
target_names = train.target_names

# --- 🔹 Limpiar documentos vacíos ---
def limpiar_docs(textos, etiquetas):
    X_limpio, y_limpio = [], []
    for x, y in zip(textos, etiquetas):
        x_clean = (x or "").strip()
        if len(x_clean) > 30:
            X_limpio.append(x_clean)
            y_limpio.append(y)
    return X_limpio, y_limpio

X_train, y_train = limpiar_docs(X_train, y_train)
X_test, y_test = limpiar_docs(X_test, y_test)

print(f"Documentos de entrenamiento luego de limpieza: {len(X_train)}")
print(f"Documentos de test luego de limpieza: {len(X_test)}")

print(f"Cantidad de documentos de entrenamiento: {len(X_train)}")
print(f"Cantidad de documentos de test: {len(X_test)}")
print(f"Cantidad de clases: {len(target_names)}")



Documentos de entrenamiento luego de limpieza: 10892
Documentos de test luego de limpieza: 7226
Cantidad de documentos de entrenamiento: 10892
Cantidad de documentos de test: 7226
Cantidad de clases: 20


In [62]:
# --------------------------------------
# 1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
#    Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el 
#    contenido del texto y la etiqueta de clasificación
# --------------------------------------

vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
X_all = vectorizer.fit_transform(X_train + X_test)
all_data = X_train + X_test
n_train = len(X_train)

# Seleccionamos 5 documentos al azar
random.seed(0)
indices = random.sample(range(X_all.shape[0]), 5)

for idx in indices:
    sim = cosine_similarity(X_all[idx], X_all).ravel()
    sim[idx] = 0  # Evitar el mismo documento
    top5 = sim.argsort()[-5:][::-1]

    # Determinar conjunto y etiqueta del documento original
    if idx < n_train:
        label = target_names[y_train[idx]]
        subset = "train"
    else:
        label = target_names[y_test[idx - n_train]]
        subset = "test"

    print("\n" + "-"*70)
    print(f"Documento índice {idx} ({subset}) — etiqueta: {label}")
    print("Extracto:", all_data[idx][:200].replace("\n", " "), "\n")

    for j in top5:
        if j < n_train:
            lab = target_names[y_train[j]]
            subset_j = "train"
        else:
            lab = target_names[y_test[j - n_train]]
            subset_j = "test"
        print(f"  • idx {j:5d} | {subset_j:5s} | label={lab:25s} | similitud={sim[j]:.4f}")




----------------------------------------------------------------------
Documento índice 12623 (test) — etiqueta: misc.forsale
Extracto: Sharp brand "Pocket Computer" model PC-1246     Dimensions;  3.5 x 5 x 0.5 inches.          Has 15-digit LCD display         53 rubber keys (w/alphabet)         built-in BASIC prog.language         an 

  • idx  4026 | train | label=rec.autos                 | similitud=0.1690
  • idx   370 | train | label=sci.electronics           | similitud=0.1644
  • idx 12058 | test  | label=comp.graphics             | similitud=0.1570
  • idx 11436 | test  | label=comp.sys.ibm.pc.hardware  | similitud=0.1494
  • idx  8680 | train | label=misc.forsale              | similitud=0.1458

----------------------------------------------------------------------
Documento índice 13781 (test) — etiqueta: misc.forsale
Extracto: Hallo all...my girlfriend and I will be travelling across the US this summer, so we won't be using our tickets to return to Hawaii.  Please buy them

El análisis de similaridad utilizando la representación TF-IDF muestra resultados coherentes en la mayoría de los casos.
Por ejemplo, los documentos con etiqueta misc.forsale presentan como más similares otros textos también pertenecientes a esa categoría, lo que tiene sentido porque contienen términos asociados a ventas (“for sale”, “buy”, “tickets”, “batch”, “copies”, “price”, etc.).
También se observa que, cuando los documentos tratan de temas más técnicos o específicos —como rec.autos o talk.politics.mideast—, las similitudes altas suelen darse con documentos de la misma clase o de clases temáticamente cercanas.
Igualmente, aparecen algunos casos en los que documentos de distintas categorías muestran similitudes no triviales. Esto se debe a que la representación TF-IDF no captura el contexto semántico profundo, sino la coocurrencia de términos. Entonces, palabras mas genéricas pueden estar presentes en múltiples categorías técnicas, afectando la medida de similitud.

In [63]:
# --------------------------------------
# 2. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto
#    de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto
#    de entrenamiento con mayor similaridad. 
# --------------------------------------

vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

y_pred_proto = []
for i in range(X_test_vec.shape[0]):
    sims = cosine_similarity(X_test_vec[i], X_train_vec).ravel()
    best = np.argmax(sims)
    y_pred_proto.append(y_train[best])

print("\n" + "="*70)
print("CLASIFICADOR POR PROTOTIPOS")
print(f"Accuracy: {accuracy_score(y_test, y_pred_proto):.4f}")
print(f"F1 macro: {f1_score(y_test, y_pred_proto, average='macro'):.4f}")



CLASIFICADOR POR PROTOTIPOS
Accuracy: 0.5569
F1 macro: 0.5492


El modelo tipo “zero-shot” basado en comparación de similaridad con los documentos de entrenamiento alcanza un accuracy de 0.56 y un F1 macro de 0.55, lo cual es razonable considerando su sencillez y la naturaleza no supervisada del enfoque.
En este tipo de modelo, cada documento de test se asigna a la clase del documento más similar en entrenamiento. Esto implica que:
Es muy sensible al "ruido" en los textos.
No aprovecha la distribución global de las clases, sino que se apoya en un único ejemplo.
Depende fuertemente de la calidad de la representación vectorial.
El rendimiento obtenido sugiere que la representación TF-IDF logra capturar cierta estructura temática, pero no lo suficiente como para generalizar bien a nuevos textos.

In [64]:
# --------------------------------------
# 3. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación 
#    (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación 
#    del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.
# --------------------------------------

configs = [
    ("CountVectorizer + MultinomialNB", CountVectorizer(stop_words='english', min_df=2), MultinomialNB()),
    ("TFIDF + MultinomialNB", TfidfVectorizer(stop_words='english', min_df=2), MultinomialNB()),
    ("TFIDF + ComplementNB", TfidfVectorizer(stop_words='english', min_df=2), ComplementNB())
]

for name, vect, model in configs:
    Xtr = vect.fit_transform(X_train)
    Xte = vect.transform(X_test)

    model.fit(Xtr, y_train)
    y_pred = model.predict(Xte)

    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average='macro')

    print("\n" + "="*70)
    print(f"MODELO: {name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 macro: {f1m:.4f}")
    print("\nReporte de clasificación (resumen):")
    print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))



MODELO: CountVectorizer + MultinomialNB
Accuracy: 0.6742
F1 macro: 0.6408

Reporte de clasificación (resumen):
                          precision    recall  f1-score   support

             alt.atheism       0.58      0.48      0.53       306
           comp.graphics       0.55      0.73      0.63       380
 comp.os.ms-windows.misc       0.33      0.00      0.01       378
comp.sys.ibm.pc.hardware       0.51      0.75      0.60       382
   comp.sys.mac.hardware       0.68      0.66      0.67       369
          comp.windows.x       0.63      0.77      0.70       382
            misc.forsale       0.84      0.74      0.79       377
               rec.autos       0.78      0.77      0.77       369
         rec.motorcycles       0.85      0.73      0.79       378
      rec.sport.baseball       0.92      0.82      0.87       374
        rec.sport.hockey       0.94      0.87      0.90       387
               sci.crypt       0.65      0.80      0.72       371
         sci.electronics     

Los resultados de los tres modelos probados muestran una mejora significativa:

Modelo	Accuracy	F1-macro
CountVectorizer + MultinomialNB	0.67	0.64
TF-IDF + MultinomialNB	0.71	0.67
TF-IDF + ComplementNB	0.74	0.71

El mejor desempeño se obtuvo con TF-IDF + ComplementNB, lo que concuerda con la teoría: el modelo ComplementNB está diseñado para manejar mejor clases desbalanceadas y situaciones donde las características mas discriminativas aparecen con baja frecuencia.

El uso de TF-IDF también mejora el rendimiento respecto a los conteos crudos, al ponderar las palabras más informativas y reducir el peso de términos comunes.
Las categorías con temas más técnicos y vocabularios más especializados (por ejemplo, rec.autos, rec.sport.hockey, sci.med, sci.space) son las mejor clasificadas, mientras que las más ambiguas o con contenido variado (talk.religion.misc, alt.atheism, talk.politics.misc) presentan menor desempeño.


In [65]:
# --------------------------------------
# 4. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento
#    que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora 
#    similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras
#    no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".
# --------------------------------------

tfidf = TfidfVectorizer(stop_words='english', min_df=5)
X_tfidf = tfidf.fit_transform(X_train)
terms = np.array(tfidf.get_feature_names_out())
# Transponer: filas = palabras, columnas = documentos
term_doc = X_tfidf.T
sim_matrix = cosine_similarity(term_doc)

selected_words = ["space", "computer", "car", "engine", "religion"]

for w in selected_words:
    if w not in tfidf.vocabulary_:
        print(f"\nPalabra '{w}' no encontrada en el vocabulario.")
        continue

    idx = tfidf.vocabulary_[w]
    sims = sim_matrix[idx]
    sims[idx] = 0
    top5 = sims.argsort()[-5:][::-1]

    print("\n" + "-"*70)
    print(f"Palabra: '{w}' — más similares:")
    for j in top5:
        print(f"  • {terms[j]:15s}  (similitud = {sims[j]:.4f})")


----------------------------------------------------------------------
Palabra: 'space' — más similares:
  • nasa             (similitud = 0.3179)
  • shuttle          (similitud = 0.2784)
  • exploration      (similitud = 0.2329)
  • aeronautics      (similitud = 0.2221)
  • sci              (similitud = 0.2167)

----------------------------------------------------------------------
Palabra: 'computer' — más similares:
  • shopper          (similitud = 0.1349)
  • verlag           (similitud = 0.1248)
  • delicate         (similitud = 0.1197)
  • drive            (similitud = 0.1106)
  • hackers          (similitud = 0.1082)

----------------------------------------------------------------------
Palabra: 'car' — más similares:
  • cars             (similitud = 0.1923)
  • dealer           (similitud = 0.1773)
  • civic            (similitud = 0.1635)
  • loan             (similitud = 0.1561)
  • owner            (similitud = 0.1484)

--------------------------------------------------

Las similitudes obtenidas son, en general, coherentes y temáticamente precisas, especialmente en dominios bien representados en el corpus (como “space”, “car” o “religion”).
En cambio, ciertos términos más genéricos o dispersos (“computer”) presentan asociaciones menos claras, lo que podría mejorarse, por ejemplo, mediante ponderación TF-IDF, reducción de dimensionalidad, o un corpus más especializado.
El análisis confirma que la transposición de la matriz y la representación término-documento permite explorar con éxito relaciones semánticas entre palabras a partir de coocurrencias contextuales.